In [2]:
# !pip freeze

In [7]:
# !pip install torch

In [27]:
# !pip install pandas
# !pip install open_clip_torch
!pip uninstall transformers --yes
!pip install transformers

In [1]:
import pandas as pd
import torch
import open_clip
import transformers

/common/home/projectgrps/IS424/IS424G20/tmp/ipykernel_2382729/873661300.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
/common/home/projectgrps/IS424/IS424G20/jupyterlab-venv-pytorch-py311/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cpu


In [3]:
df = pd.read_csv('text.csv')

In [88]:
del df['Unnamed: 0']

In [6]:
model, _, preprocess = open_clip.create_model_and_transforms('xlm-roberta-large-ViT-H-14', 'frozen_laion5b_s13b_b90k')

config.json: 100%|██████████| 616/616 [00:00<00:00, 3.50MB/s]
open_clip_pytorch_model.bin: 100%|██████████| 4.77G/4.77G [07:05<00:00, 11.2MB/s] 


In [8]:
tokenizer = open_clip.get_tokenizer('xlm-roberta-large-ViT-H-14')

sentencepiece.bpe.model: 100%|██████████| 5.07M/5.07M [00:01<00:00, 4.14MB/s]
tokenizer.json: 100%|██████████| 9.10M/9.10M [00:01<00:00, 6.19MB/s]


In [24]:
model.to(device)

CustomTextCLIP(
  (visual): VisionTransformer(
    (conv1): Conv2d(3, 1280, kernel_size=(14, 14), stride=(14, 14), bias=False)
    (patch_dropout): Identity()
    (ln_pre): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
    (transformer): Transformer(
      (resblocks): ModuleList(
        (0-31): 32 x ResidualAttentionBlock(
          (ln_1): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=1280, out_features=1280, bias=True)
          )
          (ls_1): Identity()
          (ln_2): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=1280, out_features=5120, bias=True)
            (gelu): GELU(approximate='none')
            (c_proj): Linear(in_features=5120, out_features=1280, bias=True)
          )
          (ls_2): Identity()
        )
      )
    )
    (ln_post): LayerNorm((1280,), eps=1e-05,

In [89]:
df

,label,text
0,no,As a woman you shouldnt complain about cleanin...
1,no,SimplyAddictedToGuys woof woof hot scally lad
2,no,Why the eggplant emoji doe ? y he say she look...
3,no,Baseball season for the win Yankees This is wh...
4,no,Im an early bird and Im a night owl so Im wise...
...,...,...
49671,no,This is beyond race religion and political int...
49672,no,PUBG JAPAN SERIES Grade2 Day2 Round8 CC CGR DD...
49673,no,Umar khalid sahela rashid kanhiyan kumar hate ...
49674,no,Dictator go This is the people of Iran who are...


In [101]:
def getTextFeature(text):
    with torch.no_grad(), torch.cuda.amp.autocast():
        text = tokenizer(text)
        text_features = model.encode_text(text.to(device))

    return text_features.cpu().numpy()

In [102]:
text ='hi'

with torch.no_grad(), torch.cuda.amp.autocast():
    text_features = getTextFeature(text)

In [103]:
text_features.shape

(1, 1024)

In [104]:
df['material_id'] = "text" + df.index.astype(str)

In [106]:
df.isna().sum()

Unnamed: 0     0
text           0
label          0
material_id    0
dtype: int64

In [107]:
from tqdm import tqdm

for index, row in tqdm(df.iterrows(), total=len(df)):
    try:
        text = row['text'] 
        text_feature = getTextFeature(text)
        material_id = row['material_id']
        torch.save(text_feature, f'./text_embeddings/{material_id}.pt')
    except Exception as e:
        # Handle exceptions here if needed
        print(f"Error processing row {index}: {e}")

100%|██████████| 21764/21764 [07:27<00:00, 48.63it/s]
